In [8]:
# Import des bibliothèques 
# Autograd & Numpy
import autograd
import autograd.numpy as np

# Pandas
import pandas as pd

# Matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10] # [width, height] (inches). 

# Jupyter & IPython
from IPython.display import display

In [9]:
#Définition du gradient 
def grad(f):
    g = autograd.grad
    def grad_f(x, y):
        return np.array([g(f, 0)(x, y), g(f, 1)(x, y)])
    return grad_f

In [10]:
#Définition de la Jacobienne
def J(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

In [11]:
#Implémentation de display_contour 
def display_contour(f, x, y, levels):
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    fig, ax = plt.subplots()
    contour_set = plt.contour(
        X, Y, Z, colors="grey", linestyles="dashed", 
        levels=levels 
    )
    ax.clabel(contour_set)
    plt.grid(True)
    plt.xlabel("$x_1$") 
    plt.ylabel("$x_2$")
    plt.gca().set_aspect("equal")

### Fonctions quadratiques 

In [12]:
def f1(x1, x2):
    x1 = np.array(x1)
    x2 = np.array(x2)
    return 3.0 * x1 * x1 - 2.0 * x1 * x2 + 3.0 * x2 * x2 

### Question 1

On a posé $c \in \mathbb{R}$, supposé la fonction $f:\mathbb{R}^2 \to \mathbb{R}$ continue, et telle que
$f(x_1, x_2) \to +\infty$ quand $\|(x_1,x_2)\| \to +\infty$.

Premièrement, l'ensemble de niveau $c$ de $f$ est un ensemble fermé car c'est l'image réciproque de l'ensemble fermé $\{c\}$ par une fonction continue. De plus, l'ensemble de niveau $c$ est borné donc c'est un compact car nous sommes en dimension finie. 

On a également l'équivalence entre les enoncés :

$(i)$ L'ensemble de niveau $c$ est non vide

$(ii)$ $c \ge  \underset{(x_1,x_2) \in \mathbb{R^{2}}}{\text{inf}} f(x_1,x_2)$ (Cette borne inférieure est atteinte.)



### Question 2
Au voisinage du point $x_0 = \begin{pmatrix} x_{10} \\x_{20}\\\end{pmatrix} \in \mathbb{R}^2$ où le gradient de $f$ ne s'annule pas, on pose :
$$
p(x_1, x_2) := \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|} (x_1 - x_{10}) -
\frac{\partial_1 f(x_0)}{\|\nabla f(x_0)\|} (x_2 - x_{20}).
$$

On cherche à interpréter géométriquement cette fonction $p$.

On sait que le vecteur gradient évalué en $x_0$ $\nabla f(x_0)$ est normal à la courbe de niveau en $x_0$. Ainsi, on peut former une base orthonormée en posant un vecteur normal $\overrightarrow{n}$ et un vecteur tangent $\overrightarrow{\tau}$ à la courbe de niveau en $x_0$ de la manière suivante :

$$
\overrightarrow{n}=\frac{1}{\|\nabla f(x_0)\|}
\begin{pmatrix}
\partial_1 f(x_0) \\
 \partial_2 f(x_0)\\
\end{pmatrix}
$$

$$
\overrightarrow{\tau}=\frac{1}{\|\nabla f(x_0)\|}
\begin{pmatrix}
\partial_2 f(x_0) \\
 -\partial_1 f(x_0)\\
\end{pmatrix}
$$

On se préoccupe maintenant d'un vecteur $x=\begin{pmatrix} x_1 \\x_2\\\end{pmatrix}$. On remarque que $p(x)$ n'est rien d'autre que le produit scalaire entre le vecteur $x-x_0$ et le vecteur tangentiel $\overrightarrow{\tau}$. Autrement dit, $p(x)$ est la projection du vecteur $x-x_0$ sur la tangente à la courbe de niveau en $x_0$. En effet :

$$
\left\langle x-x_0  |  \overrightarrow{\tau} \right\rangle = \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|} (x_1 - x_{10}) -
\frac{\partial_1 f(x_0)}{\|\nabla f(x_0)\|} (x_2 - x_{20})
$$

Et ainsi :

$$
p(x)=\left\langle x-x_0  |  \overrightarrow{\tau} \right\rangle
$$

In [39]:
### Implémentation python de la fonction p

def p(f,x10 ,x20 ,x1 ,x2 ):
    '''Implémentation Python de la fonction p.
    f : fonction en question
    x10, x20 : coordonnées du point x0
    x1, x2 : coordonnées d'évaluation de la fonction p'''
    grad_f = grad(f)
    gradx0= grad_f(x10,x20)
    d1=gradx0[0]
    d2=gradx0[1]
    norme=(d1*d1+d2*d2)**(1/2)
    return (d2*(x1-x10)/norme) - (d1*(x2-x20)/norme)

def affichage3d():
    fig=plt.figure()
    ax=fig.gca(projection='3d')
    x = np.linspace(-10,10.0,100)
    X, Y = np.meshgrid(x, x)
    Z = p(f1,1.0,1.0,X, Y)
    W = f1(X,Y)
    surf = ax.plot_surface(X,Y,Z)
    board = ax.plot_surface(X,Y,W)
    plt.show()


### Question 3
On pose:

$$
g : (x,t) \in  \mathbb{R^2}\times \mathbb{R} \mapsto \begin{pmatrix}
f(x) - c \\
p(x) - t
\end{pmatrix}\in \mathbb{R^2}
$$
 - g est continûment différentiable : 
$$ 
  J_g=\begin{pmatrix}
\partial_1 g_1 &\partial_2 g_1  & \partial_3 g_1  \\
 \partial_1 g_2& \partial_2 g_2 & \partial_3 g_2
\end{pmatrix}
= \begin{pmatrix}
\partial_1 f & \partial_2 f & 0 \\
 \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|}& -\frac{\partial_1 f(x_0)}{\|\nabla f(x_0)\|}  & -1
\end{pmatrix}
$$
 Effectivement, les dérivées partielles de $g_1$ et $g_2$ existent et sont continue car $f$ est elle même continûment différentiable.


 - Inversibilité de $\partial_x g$:
 

$$
\partial_x g(x,t) = \begin{pmatrix}
\partial_1f &\partial_2f  \\
\frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|}& -\frac{\partial_1 f(x_0)}{\|\nabla f(x_0)\|} 
\end{pmatrix}
$$


D'où:


$$
det (\partial_x g(x,t)) = - \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|}\partial_1f(x) - \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|}\partial_2f(x)
$$

Donc :

$$
\forall t \in \mathbb{R}, det (\partial_x g(x_0,t))= - \|\nabla f(x_0)\| \neq 0
\
\
$$


Ainsi, par continuité du déterminant, il existe $Q$ un voisinage de $x_0$ dans lequel $\partial_x g(x,t)$ est inversible pour tout t. Ainsi, il existe un ouvert  $W = Q \times \mathbb{R}$, tel que $\partial_xg$ soit inversible. 

Par le théorème des fonctions implicites il existe donc des voisinages ouverts $U$ de $x_0$ et V de $0$ tel que $0 = p(x_0)$, tels que $U \times V \subset W$ et une unique fonction $\gamma : V \to \mathbb{R^2}$ continûment différentiable telle que $\forall t \in V$ et $\forall x \in U$ :


$$
g(x,t)=0 \Leftrightarrow 
x = \gamma (t) = \gamma(p(x))
$$

Nous obtenons donc qu'il existe un $\varepsilon > 0$ et une fonction (continûment différentiable) $\gamma :\left]-\varepsilon,\varepsilon \right[ \to \mathbb{R}^2$  ($\left]-\varepsilon,\varepsilon \right[$ est un voisinage de 0) tels que dans un voisinage ouvert de $x_0,$ :
$$
f(x_1,x_2) = c \Leftrightarrow  (x_1, x_2) = \gamma(t) \ où \ t = p(x_1, x_2)
$$



### Question 4

Il s'agit d'étudier le vecteur $\gamma'(t)$. L'application du théorème des fonctions implicites donnne également :

$$
\gamma'(t) = d\gamma (t) = - (\partial_x g(x,t))^{-1} \cdot \partial_t g(x,t)
$$

Les calculs précédants permettent alors d'écrire :

$$
\gamma'(t) = - \frac{1}{det(\partial_x g(x,t))}
\begin{pmatrix}
-\frac{\partial_1 f(x_0)}{\|\nabla f(x_0)\|} & -\partial_2 f(x) \\
- \frac{\partial_2 f(x_0)}{\|\nabla f(x_0)\|} & \partial_1 f(x) \\
\end{pmatrix}

\begin{pmatrix}
0 \\
-1 \\
\end{pmatrix}
$$

$$
\gamma'(t) = \frac{1}{det(\partial_x g(x,t))}
\begin{pmatrix}
-\partial_2 f(x) \\
\partial_1 f(x)  \\
\end{pmatrix}
$$

Comme le gradient de $f$ est non-nul dans voisinage de $x_0$, on en déduit que le vecteur $\gamma'(t)$ est non nul sur ce même voisinage, c'est à dire le voisinage $V=\left] -\varepsilon; \varepsilon\right[$ introduit à la question précédente. Par ailleurs, le vecteur $\gamma'(t)$ est proportionnel au vecteur $\overrightarrow{\tau}$ tangent à la courbe, et est donc orthogonal au vecteur gradient $\Delta f(x_0)$.


### Question 5

Nous choisissons $\varepsilon = 2^{-52}$ car il s'agit de la plus petite différence entre des doubles détectable par l'ordinateur (epsilon machine).

In [33]:
N = 100
eps = 0.01

### Tâche 1
Implémentation python de la recherche de zéro par méthode de Newton pour une fonction $F:\mathbb{R}^{2}\longrightarrow \mathbb{R}^{2}$

In [34]:
def Newton(F, x0, y0, eps=eps, N=N):
    x,y=x0,y0
    X=np.array([x,y])
    J_F = J(F)
    for i in range(N):
        inv = np.linalg.inv(J_F(x,y))
        X = X - inv.dot(F(x,y))
        x,y = X[0], X[1]
        if np.sqrt((x - x0)**2 + (y - y0)**2) <= eps:
            return x, y
        x0, y0 = x, y
    else:
        raise ValueError(f"no convergence in {N} steps.")

### Tâche 2

On définit la fonction: 

$$
f_{11} : (x_1,x_2) \in  \mathbb{R^2} \mapsto 
\begin{pmatrix}
f_1(x_1,x_2) \\x_1-x_2

\end{pmatrix}
$$

Lorsque $f$ s'annule, on a alors $x_1 = x_2$.


In [45]:
def f11(x1,x2):
    f11 = np.array([f1(x1,x2), x1-x2])
    return f11

X = Newton(f11, 0.8, 0.8, eps = eps, N=N)
print (X)


(0.00625, 0.00625)
